In [1]:
import random
from collections import deque
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

Iteration = 20
gamma = 0.8 #折扣率
epsilon = 1  #Epsilon
learning_rate = 0.001
batch_size = 16
archive = deque(maxlen=100)

model = tf.keras.Sequential()
model.add(layers.Dense(16, input_dim=state_size, activation='relu'))
model.add(layers.Dense(24, activation='relu'))
model.add(layers.Dense(action_size, activation='linear')) #依照動作種類輸出
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss='mse', metrics=['mse'])

def agent(state):
    if np.random.rand() <= epsilon:
      action = np.random.choice(action_size)
    else:
      action = model.predict(state)
    return action

for i in range(Iteration):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    rewards = 0
    for t in range(100):
        action = agent(state)
        next_state, reward, done, info = env.step(action)
        archive.append((state, action, reward, next_state, done))
        rewards += reward
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state
        if done:
          print('Episode結束 {} 次, 總獎勵 {}'.format(t+1, rewards))
          break
      
        if len(archive) > batch_size:
          batch = random.sample(archive, batch_size)
          for state, action, reward, next_state, done in batch:
            if done != True:
              next_state = np.reshape(next_state, [1, state_size])
              q = learning_rate*(reward + gamma * np.amax(model.predict(next_state)[0]))
            target_all = model.predict(state)
            target_all[0][action] = q
            model.fit(state, target_all, epochs=1, verbose=0)

KeyboardInterrupt: ignored